In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import cvxpy as cp
import plotly.graph_objects as go
from tqdm.auto import tqdm
import statsmodels.api as sm
import warnings
from datetime import datetime, timedelta
from joblib import Parallel, delayed
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import time
import csv
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.timeseries import TimeSeries
import pickle


# print(plt.style.available) #list of available styles
#plt.style.use('ggplot')
# Configura el estilo de Seaborn para que los gráficos se vean más atractivos
sns.set(style="whitegrid")

plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 100
warnings.simplefilter(action='ignore', category=FutureWarning)

Cargar los tickers de los componentes del SP500 de wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
respuesta = requests.get(url)
soup = BeautifulSoup(respuesta.text, 'html.parser')

tabla = soup.find('table', {'id': 'constituents'})
filas = tabla.findAll('tr')

tickers_sp500 = []
for fila in filas[1:]:  # Excluir el encabezado de la tabla
    ticker = fila.findAll('td')[0].text.strip()
    tickers_sp500.append(ticker)

print(tickers_sp500)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DH

In [3]:
with open('../../credentials/alphavantage_key.txt', 'r') as file:
    api_key = file.read().strip()

In [5]:
fd = FundamentalData(api_key)

balances_generales = {}
resultados = {}
tickers_fallidos = []
ticker_lst = tickers_sp500.copy()

    # Establecer el intervalo de tiempo entre llamadas (2 segundos para 30 llamadas por minuto)
intervalo = 2

for ticker in tqdm(tickers_sp500):
    try:
        # Obtener la capitalización de mercado
        data, _ = fd.get_company_overview(symbol=ticker)
        market_cap = data.get('MarketCapitalization')

        # Comprobar si la capitalización de mercado está disponible
        if market_cap is None:
            raise ValueError(f"MarketCapitalization no disponible para {ticker}")

        market_cap = float(market_cap)

        # Obtener datos del balance general
        balance_sheet, _ = fd.get_balance_sheet_annual(symbol=ticker)
        
                # Almacenar en el diccionario
        balances_generales[ticker] = balance_sheet
        
        # Comprobar si hay datos en el balance general
        if balance_sheet.empty or 'totalAssets' not in balance_sheet or 'totalLiabilities' not in balance_sheet:
            raise ValueError(f"Datos del balance general no disponibles para {ticker}")

        total_assets = float(balance_sheet['totalAssets'][0])
        total_liabilities = float(balance_sheet['totalLiabilities'][0])

        # Calcular el valor en libros y la relación B/M
        book_value = total_assets - total_liabilities
        book_to_market_ratio = book_value / market_cap if market_cap else 0

        # Guardar los resultados
        resultados[ticker] = {
            "Market Capitalization": market_cap,
            "Book to Market Ratio": book_to_market_ratio
        }
    except Exception as e:
        print(f"Error al procesar {ticker}: {e}")
        tickers_fallidos.append(ticker)
        


    # Pausar el código para no exceder el límite de llamadas por minuto
    time.sleep(intervalo)

# Eliminar los tickers fallidos de la lista original
for ticker_fallido in tickers_fallidos:
    if ticker_fallido in ticker_lst:
        ticker_lst.remove(ticker_fallido)

# Resultados
print("Resultados:", resultados)
print("Tickers que fallaron y fueron eliminados:", tickers_fallidos)

  0%|          | 0/503 [00:00<?, ?it/s]

Error al procesar BRK.B: Error getting data from the api, no return was given.
Error al procesar BF.B: Error getting data from the api, no return was given.
Error al procesar DAY: Error getting data from the api, no return was given.
Resultados: {'MMM': {'Market Capitalization': 52423361000.0, 'Book to Market Ratio': 0.28174462144844165}, 'AOS': {'Market Capitalization': 11649238000.0, 'Book to Market Ratio': 0.15002698030549294}, 'ABT': {'Market Capitalization': 194629698000.0, 'Book to Market Ratio': 0.18849127536538643}, 'ABBV': {'Market Capitalization': 300141806000.0, 'Book to Market Ratio': 0.05748616039179827}, 'ACN': {'Market Capitalization': 233836675000.0, 'Book to Market Ratio': 0.10987514683058164}, 'ADBE': {'Market Capitalization': 286911529000.0, 'Book to Market Ratio': 0.05757175411379164}, 'AMD': {'Market Capitalization': 275459867000.0, 'Book to Market Ratio': 0.20290433088751908}, 'AES': {'Market Capitalization': 11397086000.0, 'Book to Market Ratio': 0.21382658690124

In [7]:
with open('../data/balances_sp500.pickle', 'rb') as file:
    balances_generales_dict = pickle.load( file)

In [17]:
balances_generales_dict['AAPL'].columns

Index(['fiscalDateEnding', 'reportedCurrency', 'totalAssets',
       'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue',
       'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables',
       'totalNonCurrentAssets', 'propertyPlantEquipment',
       'accumulatedDepreciationAmortizationPPE', 'intangibleAssets',
       'intangibleAssetsExcludingGoodwill', 'goodwill', 'investments',
       'longTermInvestments', 'shortTermInvestments', 'otherCurrentAssets',
       'otherNonCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities',
       'currentAccountsPayable', 'deferredRevenue', 'currentDebt',
       'shortTermDebt', 'totalNonCurrentLiabilities',
       'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt',
       'longTermDebtNoncurrent', 'shortLongTermDebtTotal',
       'otherCurrentLiabilities', 'otherNonCurrentLiabilities',
       'totalShareholderEquity', 'treasuryStock', 'retainedEarnings',
       'commonStock', 'commonStockSharesOutstanding'

In [18]:
# Lista para almacenar los datos para el nuevo DataFrame
data_for_balances_df = []

# Iterar a través del diccionario
for ticker, df in balances_generales_dict.items():
    # Extraer la primera fila del DataFrame y convertirla en un diccionario
    first_row = df.iloc[0].to_dict()
    # Añadir el diccionario a la lista
    data_for_balances_df.append(first_row)

# Crear el nuevo DataFrame
balances_df = pd.DataFrame(data_for_balances_df)
balances_df.index = balances_df.loc[:, 'Symbol']

balances_df

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Symbol
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,2022-12-31,USD,46455000000,14688000000,3655000000,3655000000,5372000000,4532000000,34124000000,9178000000,...,14001000000,15939000000,3190000000,5615000000,14722000000,33255000000,47950000000,9000000,549245105,MMM
AOS,2022-12-31,USD,3332300000,1633700000,391200000,481800000,516400000,581200000,1698600000,590700000,...,334500000,344500000,298400000,174600000,1747700000,1905700000,2885000000,294900000,152268436,AOS
ABT,2022-12-31,USD,74438000000,25224000000,9882000000,10170000000,6173000000,6218000000,44368000000,9162000000,...,14522000000,16773000000,8631000000,3804000000,36686000000,15229000000,35257000000,24709000000,1743373000,ABT
ABBV,2022-12-31,USD,138805000000,28463000000,9201000000,9229000000,3579000000,11254000000,111104000000,4935000000,...,None,63129000000,22468000000,30655000000,17254000000,4594000000,4784000000,18000000,1771000000,ABBV
ACN,2023-08-31,USD,51245305000,23381931000,9045032000,9049607000,None,10690713000,27863374000,1530007000,...,None,147903000,1588678000,465024000,25692839000,7062512000,19316224000,72000,629922331,ACN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2022-12-31,USD,5846000000,1609000000,367000000,367000000,442000000,680000000,4301000000,1171000000,...,None,24357000000,-734000000,1604000000,-8876000000,None,-8507000000,0,283000000,YUM
ZBRA,2022-12-31,USD,7529000000,1883000000,105000000,218000000,860000000,794000000,5646000000,278000000,...,1809000000,4055000000,882000000,108000000,2733000000,1799000000,4036000000,1000000,51668098,ZBRA
ZBH,2022-12-31,USD,21066000000,4427300000,375700000,375700000,2147200000,1381500000,16835100000,1872500000,...,5152200000,5700000000,1421300000,632600000,12020300000,6867200000,9559300000,3100000,209900000,ZBH


In [10]:
balances_generales_dict.keys()

dict_keys(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DR

Otra opción es leerlos desde un fichero

In [13]:
balances_generales_dict['AMD'].iloc[0,:]

fiscalDateEnding                           2023-12-31
reportedCurrency                                  USD
totalAssets                               67885000000
totalCurrentAssets                        16768000000
cashAndCashEquivalentsAtCarryingValue      3933000000
cashAndShortTermInvestments                5773000000
inventory                                  4351000000
currentNetReceivables                      6476000000
totalNonCurrentAssets                     51117000000
propertyPlantEquipment                     1589000000
accumulatedDepreciationAmortizationPPE     1787000000
intangibleAssets                          45625000000
intangibleAssetsExcludingGoodwill         21363000000
goodwill                                  24262000000
investments                                5253000000
longTermInvestments                          99000000
shortTermInvestments                       1840000000
otherCurrentAssets                         1268000000
otherNonCurrentAssets       

In [4]:
balance_df = pd.read_csv('../data/balances_sp500.csv', index_col=0)
balance_df


FileNotFoundError: [Errno 2] No such file or directory: '../data/balances_sp500.csv'

In [7]:
# Elimino las filas donde no haya valores de totalAssets o totalLiabilities
balance_df = balance_df.dropna(subset=['totalAssets', 'totalLiabilities'])


In [8]:
factores_df = pd.DataFrame(index=balance_df.index)

for i in balance_df.index:
   factores_df.loc[i,'pb_ratio'] = balance_df.loc[i,'totalAssets'] / balance_df.loc[i,'totalLiabilities']

In [16]:
factores_sp500 = pd.read_csv("../data/book_to_market_sp500.csv", index_col=0)
factores_sp500

,Market Capitalization,Book to Market Ratio
A,3.857104e+10,0.151539
AAL,9.149574e+09,-0.633800
AAPL,3.017854e+12,0.020593
ABBV,2.930885e+11,0.058870
ABNB,9.060771e+10,0.061363
...,...,...
YUM,3.673717e+10,-0.241608
ZBH,2.524908e+10,0.476334
ZBRA,1.315268e+10,0.207790
ZION,6.416506e+09,0.762565


In [17]:
threshold = 0.3
# Calcular los percentiles 20 y 80 para la capitalización de mercado
percentil_inf = factores_sp500['Market Capitalization'].quantile(threshold)
percentil_sup = factores_sp500['Market Capitalization'].quantile(1-threshold)

# Asignar valores a la columna SMB
factores_sp500['SMB'] = np.where(factores_sp500['Market Capitalization'] > percentil_sup, 1,
                                 np.where(factores_sp500['Market Capitalization'] < percentil_inf, -1, 0))

In [18]:
# Calcular los percentiles 20 y 80 para la capitalización de mercado
percentil_inf = factores_sp500['Book to Market Ratio'].quantile(threshold)
percentil_sup = factores_sp500['Book to Market Ratio'].quantile(1-threshold)

# Asignar valores a la columna SMB
factores_sp500['HML'] = np.where(factores_sp500['Book to Market Ratio'] > percentil_sup, 1,
                                 np.where(factores_sp500['Book to Market Ratio'] < percentil_inf, -1, 0))

In [19]:
capital_largest = factores_sp500['Market Capitalization'].nlargest(5).index
pb_largest = factores_sp500['Book to Market Ratio'].nlargest(5).index
capital_smallest = factores_sp500['Market Capitalization'].nsmallest(5).index
pb_smallest = factores_sp500['Book to Market Ratio'].nsmallest(5).index

## Resumen de los Factores Fama/French

- **Construcción de Factores Fama/French**: Utilizan 6 carteras ponderadas por valor, formadas en base al tamaño y al ratio libro-mercado.

### Factor SMB (Small Minus Big)
- Calculado como el promedio de retorno de tres carteras de empresas pequeñas menos el promedio de retorno de tres carteras de empresas grandes.
- Fórmula: 
  $ \text{SMB} = \frac{1}{3} (\text{Small Value} + \text{Small Neutral} + \text{Small Growth}) - \frac{1}{3} (\text{Big Value} + \text{Big Neutral} + \text{Big Growth}) $

### Factor HML (High Minus Low)
- Representa el retorno promedio de dos carteras de valor menos el retorno promedio de dos carteras de crecimiento.
- Fórmula: 
  $ \text{HML} = \frac{1}{2} (\text{Small Value} + \text{Big Value}) - \frac{1}{2} (\text{Small Growth} + \text{Big Growth}) $

### Rm-Rf (Exceso de Retorno del Mercado)
- Calculado como el retorno ponderado por valor de todas las firmas CRSP en EE. UU. en NYSE, AMEX o NASDAQ, menos la tasa de los bonos del Tesoro a un mes.
- Incluye todas las firmas de NYSE, AMEX y NASDAQ con datos disponibles y específicos.

### Periodo de Datos para SMB y HML
- Incluye datos de NYSE, AMEX y NASDAQ desde julio del año t hasta junio del año t+1, utilizando datos de capital de mercado de diciembre del año t-1 y datos de capital contable positivo del año t-1.

**Nota**: Para una descripción completa de los retornos de los factores, se recomienda consultar el trabajo de Fama y French de 1993, "Common Risk Factors in the Returns on Stocks and Bonds".


In [20]:
capital_smb = capital_largest.append(capital_smallest)
pb_hml = pb_largest.append(pb_smallest)

In [23]:
yf.download(list(capital_smb), start="2017-01-01", end="2023-12-31", interval="1mo")['Adj Close']

[*********************100%%**********************]  10 of 10 completed


,AAPL,AMZN,GOOG,GOOGL,HAS,MHK,MSFT,VFC,WHR,ZION
Date,,,,,,,,,,
2017-01-01,28.233551,41.174000,39.839500,41.009499,65.116890,215.839996,59.028744,38.718487,136.687820,34.696728
2017-02-01,31.872381,42.251999,41.160500,42.246498,76.924828,226.360001,58.417000,39.448029,139.579575,36.925419
2017-03-01,33.569145,44.327000,41.478001,42.389999,79.267441,229.490005,60.498096,41.343342,133.905426,34.603268
2017-04-01,33.566814,46.249500,45.298000,46.226002,78.703613,234.789993,62.886425,41.418209,145.938049,32.980213
2017-05-01,35.695553,49.730999,48.243000,49.354500,84.061577,239.300003,64.154060,40.788933,145.828033,33.013180
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,187.369781,138.009995,137.350006,136.169998,69.901978,101.389999,326.405060,19.354902,136.052628,34.670704
2023-09-01,170.984741,127.120003,131.850006,130.860001,64.212730,85.809998,315.110443,17.307749,131.664719,34.470501
2023-10-01,170.545319,133.089996,125.300003,124.080002,43.834362,80.379997,337.425140,14.657159,102.968315,30.479076


In [27]:
# Inicializa TimeSeries con tu clave API
ts = TimeSeries(key=api_key, output_format='pandas')

# Lista de tickers
tickers_smb = list(capital_smb)

# Diccionario para almacenar los DataFrames
dataframes = {}

for ticker in tickers_smb:
    # Obtiene los datos
    data, meta_data = ts.get_monthly_adjusted(symbol=ticker)
    # Almacena el precio de cierre ajustado en el diccionario
    dataframes[ticker] = data['5. adjusted close']
    
    
precios_smb = pd.DataFrame(dataframes)

precios_smb

,AAPL,MSFT,GOOGL,GOOG,AMZN,WHR,MHK,ZION,VFC,HAS
date,,,,,,,,,,
1999-12-31,0.7781,36.2649,NaN,NaN,3.8065,33.7548,26.38,38.3817,3.6514,10.2499
2000-01-31,0.7852,30.4004,NaN,NaN,3.2280,30.2216,23.44,38.5353,3.1572,8.1837
2000-02-29,0.8674,27.7601,NaN,NaN,3.4440,28.3569,22.19,34.5795,3.0051,8.5928
2000-03-31,1.0278,33.0034,NaN,NaN,3.3500,30.6125,22.38,27.1305,2.9575,9.1057
2000-04-28,0.9389,21.6658,NaN,NaN,2.7595,34.0063,24.81,27.0457,3.4725,8.7285
...,...,...,...,...,...,...,...,...,...,...
2023-10-31,170.5504,337.4254,124.08,125.30,133.0900,102.9553,80.38,30.4798,14.6573,44.5134
2023-11-30,189.9500,378.9100,132.53,133.92,146.0900,108.9000,88.31,35.6300,16.6474,45.7557
2023-12-29,192.5300,376.0400,139.69,140.93,151.9400,121.7700,103.50,43.8700,18.8000,50.3401


In [63]:
# Lista de tickers
tickers_pb = list(pb_hml)

# Diccionario para almacenar los DataFrames
dataframes = {}

for ticker in tickers_pb:
    # Obtiene los datos
    data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
    # Almacena el precio de cierre ajustado en el diccionario
    dataframes[ticker] = data['5. adjusted close']
    
    
precios_pb = pd.DataFrame(dataframes)

precios_pb

,PARA,IVZ,C,WBD,CFG,AAL,DPZ,YUM,BBWI,SBAC
date,,,,,,,,,,
1999-11-01,NaN,8.781581,223.699366,NaN,NaN,NaN,NaN,4.784646,5.523421,9.240933
1999-11-02,NaN,8.578383,224.998482,NaN,NaN,NaN,NaN,4.711508,5.427688,9.240933
1999-11-03,NaN,8.578383,222.106901,NaN,NaN,NaN,NaN,4.630112,5.315304,9.120921
1999-11-04,NaN,8.541961,229.440620,NaN,NaN,NaN,NaN,4.630112,5.211246,9.480957
1999-11-05,NaN,8.530459,227.890062,NaN,NaN,NaN,NaN,4.770490,5.202921,9.480957
...,...,...,...,...,...,...,...,...,...,...
2024-01-19,13.40,17.090000,51.520000,10.35,32.59,13.66,426.77,132.300000,42.430000,230.880000
2024-01-22,13.75,17.390000,52.840000,10.47,33.17,13.62,428.01,131.060000,42.040000,233.790000
2024-01-23,13.68,15.950000,53.300000,10.54,33.08,14.00,425.71,131.750000,41.440000,233.220000


In [50]:
precios_pb.isna().sum()

NameError: name 'precios_pb' is not defined

In [70]:
precios_smb.isna().sum()

AAPL        0
MSFT        0
GOOGL    1205
GOOG     3622
AMZN        0
WHR         0
MHK         0
ZION        0
VFC         0
HAS         0
dtype: int64

In [67]:
ff_factors = pd.read_csv('../data/F-F_Research_Data_Factors_daily.CSV', skiprows=4, index_col=0, parse_dates=True).iloc[:-1,:]

ff_factors.index = ff_factors.index.astype(str)

# Convertir el índice a formato de fecha
ff_factors.index = pd.to_datetime(ff_factors.index, format='%Y%m%d')

# Verificar el DataFrame
print(ff_factors.head())

            Mkt-RF   SMB   HML     RF
1926-07-01    0.10 -0.25 -0.27  0.009
1926-07-02    0.45 -0.33 -0.06  0.009
1926-07-06    0.17  0.30 -0.39  0.009
1926-07-07    0.09 -0.58  0.02  0.009
1926-07-08    0.21 -0.38  0.19  0.009


In [73]:
datos_df = pd.concat([ff_factors, precios_smb], axis=1, join='inner')

In [74]:
datos_df

,Mkt-RF,SMB,HML,RF,AAPL,MSFT,GOOGL,GOOG,AMZN,WHR,MHK,ZION,VFC,HAS
1999-11-01,-0.38,1.04,-0.10,0.017,0.587416,28.691986,NaN,NaN,3.4565,36.671210,23.00,38.297367,3.692251,11.190272
1999-11-02,-0.33,0.09,0.68,0.017,0.607320,28.751004,NaN,NaN,3.3220,36.929239,23.31,39.432152,3.745412,11.289496
1999-11-03,0.69,1.06,-0.70,0.017,0.616779,28.577057,NaN,NaN,3.2905,35.123030,23.94,38.945815,3.752661,11.807666
1999-11-04,0.66,-0.19,-0.47,0.017,0.632823,28.499402,NaN,NaN,3.1530,35.350096,24.00,39.315431,3.738162,12.044701
1999-11-05,0.79,-0.44,-0.26,0.017,0.668316,28.440384,NaN,NaN,3.2470,36.124186,24.69,40.327010,3.715207,12.662096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-24,0.11,0.59,0.19,0.021,189.970000,377.430000,136.69,138.22,146.7400,109.180000,84.02,34.350000,16.508097,46.150000
2023-11-27,-0.23,-0.11,-0.08,0.021,189.790000,378.610000,136.41,138.05,147.7300,108.390000,84.28,34.400000,16.498146,45.360000
2023-11-28,0.06,-0.29,0.05,0.021,190.400000,382.700000,137.20,138.62,147.0300,108.330000,85.83,34.550000,16.826517,45.710000
2023-11-29,0.01,0.44,0.69,0.021,189.370000,378.850000,134.99,136.40,146.3200,109.630000,87.80,35.440000,17.045431,46.430000


In [79]:
np.log(datos_df.loc[:, 'AAPL']).diff().dropna()

1999-11-02    0.033322
1999-11-03    0.015456
1999-11-04    0.025680
1999-11-05    0.054571
1999-11-08    0.087342
                ...   
2023-11-24   -0.007029
2023-11-27   -0.000948
2023-11-28    0.003209
2023-11-29   -0.005424
2023-11-30    0.003058
Name: AAPL, Length: 6059, dtype: float64

In [80]:
datos_df['RF']

1999-11-01    0.017
1999-11-02    0.017
1999-11-03    0.017
1999-11-04    0.017
1999-11-05    0.017
              ...  
2023-11-24    0.021
2023-11-27    0.021
2023-11-28    0.021
2023-11-29    0.021
2023-11-30    0.021
Name: RF, Length: 6060, dtype: float64

In [86]:
np.log(datos_df.loc[:, 'AAPL']).diff().dropna()

1999-11-02    0.033322
1999-11-03    0.015456
1999-11-04    0.025680
1999-11-05    0.054571
1999-11-08    0.087342
                ...   
2023-11-24   -0.007029
2023-11-27   -0.000948
2023-11-28    0.003209
2023-11-29   -0.005424
2023-11-30    0.003058
Name: AAPL, Length: 6059, dtype: float64

In [88]:
datos_df['RF'].iloc[1:]

1999-11-02    0.017
1999-11-03    0.017
1999-11-04    0.017
1999-11-05    0.017
1999-11-08    0.017
              ...  
2023-11-24    0.021
2023-11-27    0.021
2023-11-28    0.021
2023-11-29    0.021
2023-11-30    0.021
Name: RF, Length: 6059, dtype: float64

In [90]:
Y = np.log(datos_df.loc[:, 'AAPL']).diff().dropna()-datos_df['RF'].iloc[1:]

X = datos_df[['Mkt-RF', 'SMB', 'HML']].iloc[1:,:]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.319
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     946.9
Date:                Fri, 26 Jan 2024   Prob (F-statistic):               0.00
Time:                        20:09:06   Log-Likelihood:                 14474.
No. Observations:                6059   AIC:                        -2.894e+04
Df Residuals:                    6055   BIC:                        -2.891e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0058      0.000    -20.452      0.0

In [91]:
Y = np.log(datos_df.loc[:, 'AMZN']).diff().dropna()-datos_df['RF'].iloc[1:]

X = datos_df[['Mkt-RF', 'SMB', 'HML']].iloc[1:,:]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     801.4
Date:                Fri, 26 Jan 2024   Prob (F-statistic):               0.00
Time:                        20:09:55   Log-Likelihood:                 13168.
No. Observations:                6059   AIC:                        -2.633e+04
Df Residuals:                    6055   BIC:                        -2.630e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0062      0.000    -17.518      0.0